In [76]:
#pip install faker
pip install datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.8/247.8 kB 17.9 MB/s eta 0:00:00


In [81]:
import openpyxl
from faker import Faker
fake=Faker('ko_KR')

import pandas as pd
import numpy as np
import random
import time
import datetime
from random import randrange

In [ ]:
### 함수정의 ##


## 랜덤 날짜 및 시간 생성 함수
def random_datetime(start_date, end_date):
    # 시작일과 종료일 사이의 랜덤 날짜 생성
    start_timestamp = int(start_date.timestamp())
    end_timestamp = int(end_date.timestamp())
    random_timestamp = random.randint(start_timestamp, end_timestamp)
    random_datetime = datetime.datetime.fromtimestamp(random_timestamp)
    return random_datetime

# 날짜 범위 설정
start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2023, 12, 31)



## 특정 열의 값을 미리 리스트로 설정해 랜덤으로 돌릴 때, 그 비율을 설정하는 함수
def generate_column(values, probabilities, num_samples):

    if len(values) != len(probabilities):
        raise ValueError("values와 probabilities 리스트의 길이가 다릅니다.")

    if not np.isclose(sum(probabilities), 1.0):
        raise ValueError("probabilities 리스트의 총합이 1이 아닙니다.")

    # 값과 확률에 따라 랜덤 샘플 생성
    column_data = np.random.choice(values, size=num_samples, p=probabilities)
    return list(column_data)


1. 개인정보데이터셋
현재컬럼 - 이름, 주소, 직업, 나이, 월소득
    
    +대출잔액, 연체잔액, 연체횟수, 이용상품종류(예금/적금/대출), 담보자산가치, 직업안정성, 직업종류


2. 카드거래데이터셋
현재칼럼 - id, 소비 카테고리, 결제수단, 위치, 상점유형, 거래시간,  반복성여부
  
  +대출금공과금

[ 문제점  ]
1. 데이터 사이즈
  2040의 인구 수는 약 1300만명-> 1%인 13만은 해야 하지 않나?
  그렇다면, 카드거래 데이터 수는 정말 많이 커짐. .. 기간을 1년으로 설정하게 되면, 약 4천만건
2. 소비카테고리나 상점유형 등의 값을 정말 무작위로 할건지 아니면 공개데이터 가져와서 반영해서 할 건지 ! 그렇다면 좀 더 복잡해짐 하지만 의미는 있을 듯함
3. 반복성여부를 컬럼으로 넣은 이유는 그 ott나 정기구독 추가하려고 했는데 이건 좀 더 손 볼게요!

In [70]:
#개인정보 데이터셋

workbook = openpyxl.Workbook()
worksheet=workbook.active

worksheet['A1']="이름"
worksheet['B1']="주소"
worksheet['C1']="직업"
worksheet['D1']='나이'
worksheet['E1']="월소득"

fake=Faker('ko_KR')

for row in range(2,203):
  worksheet.cell(row=row, column=1, value=fake.name())
  worksheet.cell(row=row, column=2, value=fake.address())
  worksheet.cell(row=row, column=3, value=fake.job())
  worksheet.cell(row=row, column=4, value=fake.pyint(min_value=21, max_value=40))
  worksheet.cell(row=row, column=5, value=fake.pyint(min_value=50, max_value=1000))

workbook.save('test.xlsx')

In [71]:
# 개인정보 데이터셋 (이름, 주소, 직업, 나이, 월소득)

df=pd.read_excel('test.xlsx')
df=df.reset_index()
df=df.iloc[1:,:]
df=df.rename(columns={'index':'id'})
df['월소득']=df['월소득']*10000
df

,id,이름,주소,직업,나이,월소득
1,1,김예준,울산광역시 북구 서초대로,기타 주방장 및 조리사,23,6540000
2,2,최현준,부산광역시 동작구 언주131길,관세사,23,5840000
3,3,임지혜,서울특별시 양천구 양재천로,경영 및 진단 전문가,33,1330000
4,4,김민수,인천광역시 동구 학동거리,병무행정 사무원,40,4660000
5,5,문미정,세종특별자치시 서대문구 언주71길,텔레마케터,27,3070000
...,...,...,...,...,...,...
196,196,김영미,인천광역시 성북구 백제고분가 (정호이읍),비서,36,4840000
197,197,김명자,강원도 청주시 청원구 강남대2길 (순옥최면),유치원 교사,37,7630000
198,198,최명자,대구광역시 서구 강남대길 (선영최박읍),고객 상담 및 모니터 요원,39,9020000
199,199,김현숙,광주광역시 영등포구 백제고분거리 (예원이김동),자동차 영업원,30,610000


In [99]:
df['나이'].value_counts()

나이
33    19
30    17
26    14
38    12
37    11
29    10
34    10
23     9
28     9
36     9
31     9
32     9
24     9
40     9
39     8
27     8
21     8
22     7
35     7
25     6
Name: count, dtype: int64

In [95]:
#카드거래 데이터셋(id, 소비 카테고리, 결제수단, 위치, 상점유형, 거래시간, 반복성 여부)

workbook2 = openpyxl.Workbook()
worksheet2=workbook2.active

name_list=df['이름'].tolist()
worksheet2['A1']="id"
worksheet2['B1']="소비 카테고리"
worksheet2['C1']="결제수단"
worksheet2['D1']='위치'
worksheet2['E1']="상점유형"
worksheet2['F1']="거래시간"
worksheet2['G1']="반복성 여부"

for row in range(2,2000):
  rand_datetime=random_datetime(start_date, end_date)
  worksheet2.cell(row=row, column=1, value=random.choice(name_list))
  worksheet2.cell(row=row, column=2, value=random.choice(['식비','쇼핑','교통비','생활비, ']))
  worksheet2.cell(row=row, column=3, value=random.choice(['신용카드','체크카드','모바일 결제']))
  worksheet2.cell(row=row, column=4, value=fake.address())
  worksheet2.cell(row=row, column=5, value=random.choice(['온라인','오프라인']))
  worksheet2.cell(row=row, column=6, value=rand_datetime.strftime('%Y-%m-%d %H:%M:%S'))
  worksheet2.cell(row=row, column=7, value=random.choice([0,1]))

workbook2.save('test.xlsx')

In [96]:
# 카드거래 데이터셋 (id, 소비 카테고리, 결제수단, 위치, 상점유형, 거래시간, 반복성 여부)

df2=pd.read_excel('test.xlsx')
df2=df2.reset_index()
df2

,index,id,소비 카테고리,결제수단,위치,상점유형,거래시간,반복성 여부
0,0,안영환,쇼핑,모바일 결제,경상북도 안산시 상록구 논현길 (은지박심마을),온라인,2023-06-07 03:48:40,0
1,1,홍현숙,쇼핑,체크카드,부산광역시 노원구 테헤란거리,온라인,2023-04-23 04:30:31,0
2,2,최지원,식비,신용카드,인천광역시 중구 테헤란길 (민서정김마을),온라인,2023-06-24 17:50:14,1
3,3,안민수,식비,신용카드,광주광역시 양천구 백제고분6로,오프라인,2023-01-18 22:38:35,0
4,4,한경자,교통비,체크카드,울산광역시 은평구 강남대275가,온라인,2023-10-03 02:58:10,1
...,...,...,...,...,...,...,...,...
1993,1993,배주원,쇼핑,신용카드,전라북도 오산시 서초중앙로 (하은박리),오프라인,2023-05-13 21:21:34,1
1994,1994,김지아,"생활비,",신용카드,대구광역시 동작구 압구정가 (혜진심이면),온라인,2023-11-03 05:55:59,1
1995,1995,이보람,쇼핑,모바일 결제,서울특별시 은평구 서초중앙9거리 (진우이서읍),오프라인,2023-10-13 21:40:47,1
1996,1996,장은주,식비,신용카드,서울특별시 송파구 학동로,온라인,2023-08-12 13:06:24,1


In [ ]:
소비카테고리 생각해본 것)

통신비
아파트관리비
구독서비스(OTT, 배달어플)
학원비
독서실
스포츠(운동시설, 레저용품 등)
자동차관련(주유비, 자동차 용품 등)
쇼핑(백화점, 아울렛, 마트, 재래시장, 온라인 등)
놀이공원(에버랜드, 롯데월드, 케리비안베이, 워터파크)
숙박비(호텔, 모텔, 여관 등)
항공비
미용비(미용실)
패션(의류, 시계, 귀금속 ,안경 등)
게임(온라인 캐시)
유흥비(영화, 공연, 노래방 등)
기타 가전/가구
세탁소
서점
병원(종합병원, 한의원, 일반병원, 동물병원 등)